In [3]:
import pandas as pd


# Reload the data while skipping the first row which seems to contain redundant headers
data = pd.read_csv('Homework6_data.csv', skiprows=1)
data.head()

,Region1,Region2,Region3,Region4,Region1.1,Region2.1,Region3.1,Region4.1,Region1.2,Region2.2,Region3.2,Region4.2
0,28.97,28.04,32.91,25.39,9.67,5.64,8.74,10.90,11.76,8.88,13.25,13.25
1,32.27,29.22,32.37,23.99,10.73,10.53,14.25,18.67,9.00,10.45,8.67,11.30
2,24.25,27.95,37.98,19.19,12.36,10.12,8.64,9.43,8.31,10.83,11.02,9.19
3,28.44,34.83,28.68,30.89,13.71,12.60,10.24,7.07,9.97,8.69,9.40,13.39
4,35.25,35.53,32.67,25.56,7.27,14.27,6.57,10.88,9.95,10.68,14.84,12.62


In [4]:
# Rename columns for clarity
data.columns = ['Product1_Region1', 'Product1_Region2', 'Product1_Region3', 'Product1_Region4',
                'Product2_Region1', 'Product2_Region2', 'Product2_Region3', 'Product2_Region4',
                'Product3_Region1', 'Product3_Region2', 'Product3_Region3', 'Product3_Region4']

# Isolate Product 1 data
product1_data = data[['Product1_Region1', 'Product1_Region2', 'Product1_Region3', 'Product1_Region4']]

# Calculate mean and standard deviation for each region for Product 1
mean_demand = product1_data.mean()
std_demand = product1_data.std()

mean_demand, std_demand


(Product1_Region1    29.875534
 Product1_Region2    29.833781
 Product1_Region3    30.155616
 Product1_Region4    29.666055
 dtype: float64,
 Product1_Region1    4.896481
 Product1_Region2    5.255072
 Product1_Region3    4.936236
 Product1_Region4    4.949998
 dtype: float64)

In [5]:
from scipy.stats import norm

# Constants
review_period = 6
lead_time = 5
service_level = 0.95
z_score = norm.ppf(service_level)
unit_holding_cost = 0.15
inbound_cost = 0.09
outbound_cost = 0.10

# Calculate Order Up-To Level (OUL) and other metrics for each region
inventory_metrics = {}

for region in ['Product1_Region1', 'Product1_Region2', 'Product1_Region3', 'Product1_Region4']:
    demand_mean = mean_demand[region]
    demand_std = std_demand[region]

    # Average demand during lead time + review period
    avg_demand_lead_review = demand_mean * (lead_time + review_period)

    # Safety stock
    safety_stock = z_score * demand_std * (lead_time + review_period)**0.5

    # Order up-to level
    oul = avg_demand_lead_review + safety_stock

    # Average order quantity
    average_order_quantity = demand_mean * review_period

    # Average cycle stock
    average_cycle_stock = average_order_quantity / 2

    # Average inventory
    average_inventory = average_cycle_stock + safety_stock

    # Daily average inventory holding cost
    daily_holding_cost = average_inventory * unit_holding_cost

    # Daily average transportation cost
    daily_transport_cost = (inbound_cost + outbound_cost) * demand_mean

    # Sum of daily costs
    total_daily_cost = daily_holding_cost + daily_transport_cost

    inventory_metrics[region] = {
        'OUL': oul,
        'Average Order Quantity': average_order_quantity,
        'Average Cycle Stock': average_cycle_stock,
        'Average Safety Stock': safety_stock,
        'Average Inventory': average_inventory,
        'Daily Average Inventory Holding Cost': daily_holding_cost,
        'Daily Average Transportation Cost': daily_transport_cost,
        'Total Daily Cost': total_daily_cost
    }

inventory_metrics


{'Product1_Region1': {'OUL': 355.3429572255559,
  'Average Order Quantity': 179.25320547945205,
  'Average Cycle Stock': 89.62660273972602,
  'Average Safety Stock': 26.712080513227143,
  'Average Inventory': 116.33868325295316,
  'Daily Average Inventory Holding Cost': 17.450802487942973,
  'Daily Average Transportation Cost': 5.676351506849315,
  'Total Daily Cost': 23.127153994792288},
 'Product1_Region2': {'OUL': 356.83991227111534,
  'Average Order Quantity': 179.00268493150685,
  'Average Cycle Stock': 89.50134246575342,
  'Average Safety Stock': 28.66832323001946,
  'Average Inventory': 118.16966569577289,
  'Daily Average Inventory Holding Cost': 17.725449854365934,
  'Daily Average Transportation Cost': 5.668418356164384,
  'Total Daily Cost': 23.393868210530318},
 'Product1_Region3': {'OUL': 358.6407369473133,
  'Average Order Quantity': 180.93369863013697,
  'Average Cycle Stock': 90.46684931506849,
  'Average Safety Stock': 26.928956125395512,
  'Average Inventory': 117.395

One Regional Center

In [6]:
# Calculate total average demand across all regions for Product 1
total_average_demand = product1_data.mean(axis=1).mean()
total_std_demand = product1_data.stack().std()  # Standard deviation across all data points

# Calculations for the national distribution center
# Average demand during lead time + review period
avg_demand_lead_review_national = total_average_demand * (lead_time + review_period)

# Safety stock for national distribution center
safety_stock_national = z_score * total_std_demand * (lead_time + review_period)**0.5

# Order up-to level for national distribution center
oul_national = avg_demand_lead_review_national + safety_stock_national

# Average order quantity for national distribution center
average_order_quantity_national = total_average_demand * review_period

# Average cycle stock for national distribution center
average_cycle_stock_national = average_order_quantity_national / 2

# Average inventory for national distribution center
average_inventory_national = average_cycle_stock_national + safety_stock_national

# Daily average inventory holding cost for national distribution center
daily_holding_cost_national = average_inventory_national * unit_holding_cost

# Daily average transportation cost for national distribution center (new costs)
daily_transport_cost_national = (0.05 + 0.24) * total_average_demand

# Sum of daily costs for national distribution center
total_daily_cost_national = daily_holding_cost_national + daily_transport_cost_national

{
    'OUL': oul_national,
    'Average Order Quantity': average_order_quantity_national,
    'Average Cycle Stock': average_cycle_stock_national,
    'Average Safety Stock': safety_stock_national,
    'Average Inventory': average_inventory_national,
    'Daily Average Inventory Holding Cost': daily_holding_cost_national,
    'Daily Average Transportation Cost': daily_transport_cost_national,
    'Total Daily Cost': total_daily_cost_national
}


{'OUL': 356.0384746334782,
 'Average Order Quantity': 179.29647945205477,
 'Average Cycle Stock': 89.64823972602738,
 'Average Safety Stock': 27.328262304711064,
 'Average Inventory': 116.97650203073844,
 'Daily Average Inventory Holding Cost': 17.546475304610766,
 'Daily Average Transportation Cost': 8.665996506849314,
 'Total Daily Cost': 26.21247181146008}

Product2

In [7]:
# Isolate Product 2 data
product2_data = data[['Product2_Region1', 'Product2_Region2', 'Product2_Region3', 'Product2_Region4']]

# Calculate mean and standard deviation for each region for Product 2
mean_demand_product2 = product2_data.mean()
std_demand_product2 = product2_data.std()

# Calculate inventory metrics for each region for Product 2
inventory_metrics_product2 = {}

for region in ['Product2_Region1', 'Product2_Region2', 'Product2_Region3', 'Product2_Region4']:
    demand_mean = mean_demand_product2[region]
    demand_std = std_demand_product2[region]

    # Average demand during lead time + review period
    avg_demand_lead_review = demand_mean * (lead_time + review_period)

    # Safety stock
    safety_stock = z_score * demand_std * (lead_time + review_period)**0.5

    # Order up-to level
    oul = avg_demand_lead_review + safety_stock

    # Average order quantity
    average_order_quantity = demand_mean * review_period

    # Average cycle stock
    average_cycle_stock = average_order_quantity / 2

    # Average inventory
    average_inventory = average_cycle_stock + safety_stock

    # Daily average inventory holding cost
    daily_holding_cost = average_inventory * unit_holding_cost

    # Daily average transportation cost
    daily_transport_cost = (inbound_cost + outbound_cost) * demand_mean

    # Sum of daily costs
    total_daily_cost = daily_holding_cost + daily_transport_cost

    inventory_metrics_product2[region] = {
        'OUL': oul,
        'Average Order Quantity': average_order_quantity,
        'Average Cycle Stock': average_cycle_stock,
        'Average Safety Stock': safety_stock,
        'Average Inventory': average_inventory,
        'Daily Average Inventory Holding Cost': daily_holding_cost,
        'Daily Average Transportation Cost': daily_transport_cost,
        'Total Daily Cost': total_daily_cost
    }

# Calculate total average demand across all regions for Product 2 for national distribution center
total_average_demand_product2 = product2_data.mean(axis=1).mean()
total_std_demand_product2 = product2_data.stack().std()  # Standard deviation across all data points

# Calculations for the national distribution center for Product 2
# Average demand during lead time + review period
avg_demand_lead_review_national = total_average_demand_product2 * (lead_time + review_period)

# Safety stock for national distribution center
safety_stock_national = z_score * total_std_demand_product2 * (lead_time + review_period)**0.5

# Order up-to level for national distribution center
oul_national = avg_demand_lead_review_national + safety_stock_national

# Average order quantity for national distribution center
average_order_quantity_national = total_average_demand_product2 * review_period

# Average cycle stock for national distribution center
average_cycle_stock_national = average_order_quantity_national / 2

# Average inventory for national distribution center
average_inventory_national = average_cycle_stock_national + safety_stock_national

# Daily average inventory holding cost for national distribution center
daily_holding_cost_national = average_inventory_national * unit_holding_cost

# Daily average transportation cost for national distribution center (new costs)
daily_transport_cost_national = (0.05 + 0.24) * total_average_demand_product2

# Sum of daily costs for national distribution center
total_daily_cost_national = daily_holding_cost_national + daily_transport_cost_national

inventory_metrics_product2['National'] = {
    'OUL': oul_national,
    'Average Order Quantity': average_order_quantity_national,
    'Average Cycle Stock': average_cycle_stock_national,
    'Average Safety Stock': safety_stock_national,
    'Average Inventory': average_inventory_national,
    'Daily Average Inventory Holding Cost': daily_holding_cost_national,
    'Daily Average Transportation Cost': daily_transport_cost_national,
    'Total Daily Cost': total_daily_cost_national
}

inventory_metrics_product2


{'Product2_Region1': {'OUL': 129.25092834766178,
  'Average Order Quantity': 61.73490410958904,
  'Average Cycle Stock': 30.86745205479452,
  'Average Safety Stock': 16.07027081341521,
  'Average Inventory': 46.93772286820973,
  'Daily Average Inventory Holding Cost': 7.04065843023146,
  'Daily Average Transportation Cost': 1.9549386301369862,
  'Total Daily Cost': 8.995597060368446},
 'Product2_Region2': {'OUL': 128.17999257613104,
  'Average Order Quantity': 61.005205479452044,
  'Average Cycle Stock': 30.502602739726022,
  'Average Safety Stock': 16.337115863802275,
  'Average Inventory': 46.8397186035283,
  'Daily Average Inventory Holding Cost': 7.025957790529245,
  'Daily Average Transportation Cost': 1.9318315068493148,
  'Total Daily Cost': 8.95778929737856},
 'Product2_Region3': {'OUL': 124.7456083327847,
  'Average Order Quantity': 59.348054794520536,
  'Average Cycle Stock': 29.674027397260268,
  'Average Safety Stock': 15.940841209497037,
  'Average Inventory': 45.614868606

Product 3

In [8]:
# Isolate Product 3 data
product3_data = data[['Product3_Region1', 'Product3_Region2', 'Product3_Region3', 'Product3_Region4']]

# Calculate mean and standard deviation for each region for Product 3
mean_demand_product3 = product3_data.mean()
std_demand_product3 = product3_data.std()

# Calculate inventory metrics for each region for Product 3
inventory_metrics_product3 = {}

for region in ['Product3_Region1', 'Product3_Region2', 'Product3_Region3', 'Product3_Region4']:
    demand_mean = mean_demand_product3[region]
    demand_std = std_demand_product3[region]

    # Average demand during lead time + review period
    avg_demand_lead_review = demand_mean * (lead_time + review_period)

    # Safety stock
    safety_stock = z_score * demand_std * (lead_time + review_period)**0.5

    # Order up-to level
    oul = avg_demand_lead_review + safety_stock

    # Average order quantity
    average_order_quantity = demand_mean * review_period

    # Average cycle stock
    average_cycle_stock = average_order_quantity / 2

    # Average inventory
    average_inventory = average_cycle_stock + safety_stock

    # Daily average inventory holding cost
    daily_holding_cost = average_inventory * unit_holding_cost

    # Daily average transportation cost
    daily_transport_cost = (inbound_cost + outbound_cost) * demand_mean

    # Sum of daily costs
    total_daily_cost = daily_holding_cost + daily_transport_cost

    inventory_metrics_product3[region] = {
        'OUL': oul,
        'Average Order Quantity': average_order_quantity,
        'Average Cycle Stock': average_cycle_stock,
        'Average Safety Stock': safety_stock,
        'Average Inventory': average_inventory,
        'Daily Average Inventory Holding Cost': daily_holding_cost,
        'Daily Average Transportation Cost': daily_transport_cost,
        'Total Daily Cost': total_daily_cost
    }

# Calculate total average demand across all regions for Product 3 for national distribution center
total_average_demand_product3 = product3_data.mean(axis=1).mean()
total_std_demand_product3 = product3_data.stack().std()  # Standard deviation across all data points

# Calculations for the national distribution center for Product 3
# Average demand during lead time + review period
avg_demand_lead_review_national = total_average_demand_product3 * (lead_time + review_period)

# Safety stock for national distribution center
safety_stock_national = z_score * total_std_demand_product3 * (lead_time + review_period)**0.5

# Order up-to level for national distribution center
oul_national = avg_demand_lead_review_national + safety_stock_national

# Average order quantity for national distribution center
average_order_quantity_national = total_average_demand_product3 * review_period

# Average cycle stock for national distribution center
average_cycle_stock_national = average_order_quantity_national / 2

# Average inventory for national distribution center
average_inventory_national = average_cycle_stock_national + safety_stock_national

# Daily average inventory holding cost for national distribution center
daily_holding_cost_national = average_inventory_national * unit_holding_cost

# Daily average transportation cost for national distribution center (new costs)
daily_transport_cost_national = (0.05 + 0.24) * total_average_demand_product3

# Sum of daily costs for national distribution center
total_daily_cost_national = daily_holding_cost_national + daily_transport_cost_national

inventory_metrics_product3['National'] = {
    'OUL': oul_national,
    'Average Order Quantity': average_order_quantity_national,
    'Average Cycle Stock': average_cycle_stock_national,
    'Average Safety Stock': safety_stock_national,
    'Average Inventory': average_inventory_national,
    'Daily Average Inventory Holding Cost': daily_holding_cost_national,
    'Daily Average Transportation Cost': daily_transport_cost_national,
    'Total Daily Cost': total_daily_cost_national
}

inventory_metrics_product3


{'Product3_Region1': {'OUL': 124.55834341890922,
  'Average Order Quantity': 58.68706849315069,
  'Average Cycle Stock': 29.343534246575345,
  'Average Safety Stock': 16.96538451479962,
  'Average Inventory': 46.30891876137497,
  'Daily Average Inventory Holding Cost': 6.946337814206245,
  'Daily Average Transportation Cost': 1.8584238356164384,
  'Total Daily Cost': 8.804761649822684},
 'Product3_Region2': {'OUL': 123.14937878049689,
  'Average Order Quantity': 58.05156164383561,
  'Average Cycle Stock': 29.025780821917806,
  'Average Safety Stock': 16.721515766798262,
  'Average Inventory': 45.74729658871607,
  'Daily Average Inventory Holding Cost': 6.86209448830741,
  'Daily Average Transportation Cost': 1.8382994520547946,
  'Total Daily Cost': 8.700393940362204},
 'Product3_Region3': {'OUL': 126.48644078767994,
  'Average Order Quantity': 58.87446575342466,
  'Average Cycle Stock': 29.43723287671233,
  'Average Safety Stock': 18.54992023973473,
  'Average Inventory': 47.987153116